# TU Delft - EPA1333 - Computer Engineering for Scientific Computing - 2017

## Final assignment


Group 40:

Luca de Laat 4135040

Pawel Marciniak 4633482

### WORK CELL - to be removed before submission
"analyse which countries are most effecient at reducing CO2 emission/deforestation/overfishing... while staying competitive in the global market. The idea is to find which countries have the most effective policy for fighting climate change at the lowest cost.we'd need to find some kind of index or define one ourselves."

In [39]:
import pandas as pd
import requests
import xml.etree.ElementTree as etree
from pandas_datareader import wb

In [40]:
#INDICATORS WE CAN USE

# EN.ATM.GHGT.KT.CE Total greenhouse gas emissions (kt of CO2 equivalent) 

# EN.ATM.CO2E.PC    CO2 emissions (metric tons per capita)
# http://databank.worldbank.org/data/reports.aspx?source=2&type=metadata&series=EN.ATM.CO2E.PC

# NY.GDP.MKTP.CD    GDP (in current US$)

# EG.GDP.PUSE.KO.PP.KD GDP per unit of energy use (constant 2011 PPP $ per kg of oil equivalent)
# http://databank.worldbank.org/data/reports.aspx?source=2&type=metadata&series=EG.GDP.PUSE.KO.PP.KD

# EG.USE.PCAP.KG.OE Energy use (kg of oil equivalent per capita) 

# EG.FEC.RNEW.ZS    Renewable energy consumption (% of total final energy consumption)

# EG.ELC.RNWX.KH    Electricity production from renewable sources, excluding hydroelectric (kWh)

# EG.USE.COMM.CL.ZS Alternative and nuclear energy (% of total energy use)
# http://databank.worldbank.org/data/reports.aspx?source=2&type=metadata&series=EG.USE.COMM.CL.ZS

# EG.ELC.RNEW.ZS    Renewable electricity output (% of total electricity output)
# http://databank.worldbank.org/data/reports.aspx?source=2&type=metadata&series=EG.ELC.RNEW.ZS

# IQ.CPA.ENRV.XQ    CPIA policy and institutions for environmental sustainability rating (1=low to 6=high)
# http://databank.worldbank.org/data/reports.aspx?source=2&type=metadata&series=IQ.CPA.ENVR.XQ

# SP.POP.GROW       Population growth (annual %)
# http://databank.worldbank.org/data/reports.aspx?source=2&type=metadata&series=SP.POP.GROW

We explored World Bank data bases and selected those indicators which we found interesting for our report. The full list of indicators (i.e. data items collected by World Bank) is available through this api link:

http://api.worldbank.org/indicators

A more convenient way to explore available data is to download the relevant csv / Excel files with list of indicators from the https://data.worldbank.org/data-catalog/ pages, and then reviewing these files to identify interesting indicators to explore. This is what we did. We explored the following sets of indicators:

https://data.worldbank.org/data-catalog/climate-change
https://data.worldbank.org/data-catalog/world-development-indicators

and create the following list of selected indicators to use for our analysis:

In [41]:
# creating list of interesting indicators 
# we are going to download all data for these indicators directly from World Bank DB

selectedInd = [
    'EN.ATM.GHGT.KT.CE', # Total greenhouse gas emissions (kt of CO2 equivalent)
    'EN.ATM.CO2E.PC',    # CO2 emissions (metric tons per capita)
    'NY.GDP.MKTP.CD'     # GDP (in current US$)
]

# TO DO: complete this list!

In [42]:
# initializing an empty pandas data frame 
# we need this if we want to come back to this cell from below to re-download the data 
df_raw = pd.DataFrame() 
df_raw.info()

# downloading data for the first of the indicators
# if we try to download all the data at once, there are problems with reindexing of the data frame
print('\n')
df_raw = wb.download(indicator=selectedInd[0], country=['all'], start=1988, end=2017)
print('WB data: downloaded: ', selectedInd[0], '\n')
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame

WB data: downloaded:  EN.ATM.GHGT.KT.CE 

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7656 entries, (Arab World, 2016) to (Zimbabwe, 1988)
Data columns (total 1 columns):
EN.ATM.GHGT.KT.CE    5752 non-null float64
dtypes: float64(1)
memory usage: 84.7+ KB


In [43]:
# now downloading data for the remaining indicators

for x in range(1, len(selectedInd)):
    df_raw[selectedInd[x]] = wb.download(indicator = selectedInd[x], country=['all'], start=1988, end=2017)
    print('WB data: downloaded: ', selectedInd[x])

print('\n')
df_raw.info()

WB data: downloaded:  EN.ATM.CO2E.PC
WB data: downloaded:  NY.GDP.MKTP.CD


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7656 entries, (Arab World, 2016) to (Zimbabwe, 1988)
Data columns (total 3 columns):
EN.ATM.GHGT.KT.CE    5752 non-null float64
EN.ATM.CO2E.PC       6503 non-null float64
NY.GDP.MKTP.CD       6903 non-null float64
dtypes: float64(3)
memory usage: 204.3+ KB


In [44]:
# Let's see how this data frame is indexed
df_raw.index.names

FrozenList(['country', 'year'])

In [45]:
# We noticed that World Bank uses groupings of countries. Let's see what values we have in the corresponding index item:

wbCountries = list(df_raw.index.get_level_values('country').unique())
wbCountries

['Arab World',
 'Caribbean small states',
 'Central Europe and the Baltics',
 'Early-demographic dividend',
 'East Asia & Pacific',
 'East Asia & Pacific (excluding high income)',
 'East Asia & Pacific (IDA & IBRD countries)',
 'Euro area',
 'Europe & Central Asia',
 'Europe & Central Asia (excluding high income)',
 'Europe & Central Asia (IDA & IBRD countries)',
 'European Union',
 'Fragile and conflict affected situations',
 'Heavily indebted poor countries (HIPC)',
 'High income',
 'IBRD only',
 'IDA & IBRD total',
 'IDA blend',
 'IDA only',
 'IDA total',
 'Late-demographic dividend',
 'Latin America & Caribbean',
 'Latin America & Caribbean (excluding high income)',
 'Latin America & the Caribbean (IDA & IBRD countries)',
 'Least developed countries: UN classification',
 'Low & middle income',
 'Low income',
 'Lower middle income',
 'Middle East & North Africa',
 'Middle East & North Africa (excluding high income)',
 'Middle East & North Africa (IDA & IBRD countries)',
 'Middle inc

If we only want to analyse data on the country level, and ditch the groupings data, we need to take data starting with 'Afghanistan', so let's take its index and slice the list to create the list containing countries only:

In [46]:
selected_wbCountries= wbCountries[wbCountries.index('Afghanistan'):]
selected_wbCountries

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas, The',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Channel Islands',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Curacao',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt, Arab Rep.',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Faroe Islands',
 'Fiji',
 'Finland',
 'France',
 'French Pol

In [102]:
# subsetting our data frame to data with only the selected countries:

df_raw.sort_index(inplace=True)
idx = pd.IndexSlice
df = df_raw.loc[idx[selected_wbCountries, :], :]#.copy()
df

EN.ATM.GHGT.KT.CE  EN.ATM.CO2E.PC  NY.GDP.MKTP.CD
country     year                                                   
Afghanistan 1988       12959.294800        0.248473             NaN
            1989       12348.957000        0.235695             NaN
            1990       12528.016500        0.213450             NaN
            1991       12307.063800        0.187673             NaN
            1992       11370.453100        0.099666             NaN
            1993       11390.874800        0.089154             NaN
            1994       11290.688200        0.080039             NaN
            1995       11635.659400        0.072699             NaN
            1996       12505.930800        0.066045             NaN
            1997       13343.250000        0.059648             NaN
            1998       13793.367000        0.055207             NaN
            1999       14812.366000        0.042333             NaN
            2000       13530.953000        0.038506             NaN
            2001       12585.402000        0.039002    2.461666e+09
            2002       13842.738100        0.048716    4.128821e+09
            2003       14709.884000        0.051830    4.583644e+09
            2004       14654.374000        0.039378    5.285466e+09
            2005       14770.264300        0.052948    6.275074e+09
            2006       14849.159400        0.063728    7.057598e+09
            2007       15272.633000        0.085418    9.843842e+09
            2008       15288.775600        0.154101    1.019053e+10
            2009       15554.429500        0.241723    1.248694e+10
            2010       17791.329902        0.293837    1.593680e+10
            2011       17981.307287        0.412017    1.793024e+10
            2012                NaN        0.350371    2.053654e+10
            2013                NaN        0.315602    2.004633e+10
            2014                NaN        0.299445    2.005019e+10
            2015                NaN             NaN    1.970299e+10
            2016                NaN             NaN    1.946902e+10
Albania     1988       12603.886700        2.331599    2.126000e+09
...                             ...             ...             ...
Zambia      2016                NaN             NaN    1.955109e+10
Zimbabwe    1988       29369.643000        1.676519    7.814784e+09
            1989       30264.471000        1.634544    8.286323e+09
            1990       35100.590000        1.536212    8.783817e+09
            1991       34132.529000        1.527823    8.641482e+09
            1992       38275.978000        1.592383    6.751472e+09
            1993       30183.416000        1.500323    6.563813e+09
            1994       30208.734000        1.600795    6.890675e+09
            1995       30376.986000        1.336535    7.111271e+09
            1996       30570.239000        1.301790    8.553147e+09
            1997       29469.426000        1.230371    8.529572e+09
            1998       76391.804000        1.198462    6.401968e+09
            1999       66724.652000        1.312450    6.858013e+09
            2000       51434.535000        1.139201    6.689958e+09
            2001       58452.519000        1.018598    6.777385e+09
            2002       50281.895000        0.957193    6.342116e+09
            2003       48216.933000        0.843469    5.727592e+09
            2004       61742.544000        0.742440    5.805598e+09
            2005       59894.603000        0.832299    5.755215e+09
            2006       55535.280000        0.796028    5.443896e+09
            2007       60917.825000        0.741658    5.291950e+09
            2008       68840.706000        0.573101    4.415703e+09
            2009       67914.131008        0.405716    8.366794e+09
            2010       71019.116728        0.552146    1.005205e+10
            2011       71561.952250        0.664751    1.207173e+10
            2012       72057.803322        0.529703    1.405838e+10
      

hello
